In [53]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pred_utils import interpret_lime_results_v5, cfe_live, cf_explanations, get_data, visualize_counterfactuals_radar_plotly, visualize_counterfactuals_plotly, explain_counterfactual_percentage, generate_transaction_features, main_instance, cfe_instance
import pandas as pd
import joblib
import time
import traceback
#from joblib import load 
X_train_ad, y_train_ad, X_train_smote, y_train_smote, X_train_stomek, y_train_stomek = get_data()

In [54]:
# import pandas as pd
# import numpy as np
# import fraud_utils
# from fraud_utils import *
# import joblib 
# from joblib import load 


# if model_selection == 'Random Forest':
#     model = joblib.load('rfc_pipline_a.pkl')    
# elif model_selection == 'Gradient Boost':    
#     model = joblib.load('gbc_pipline_s.pkl') 
# elif model_selection == 'Neural Network':    
#     model = joblib.load('mlp_pipline_stm.pkl')
# else:
#     print('No Model selected')
    
rfc_pipeline = load('rfc_adasyn_pipeline.pkl')
gbc_pipeline = load('gbc_smote_pipeline.pkl')
mlp_pipeline = load('mlp_smotetomek_pipeline.pkl')
gbc_pipeline1 = load('gbc_smotetomek_pipeline.pkl')

User_id = 'C3562'
transaction_type = 1
amount = 6509
step = 170
oldbalanceOrg = 578




newbalanceOrig = oldbalanceOrg + amount
# Create DataFrame with a dictionary


    
df = pd.DataFrame({'step': [step], 'type': [transaction_type], 'amount': [amount], 'oldbalanceOrg': [oldbalanceOrg], 'newbalanceOrig': [newbalanceOrig]})

live_df = generate_transaction_features(df)
# Drop the 'newbalanceOrig' column
#live_df.drop(columns='newbalanceOrig', axis=1, inplace=True)


In [51]:
live_df

,step,type,amount,oldbalanceOrg,bal_chg,orig_zero,amt_bal_ratio,chg_amt_ratio
0,170,1,6509,578,6509,0,11.261246,1.0


In [39]:
live_df.columns

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'bal_chg', 'orig_zero',
       'amt_bal_ratio', 'chg_amt_ratio'],
      dtype='object')

In [52]:
pred = mlp_pipeline.predict(live_df)
prediction = pred[0]
prediction

1

In [41]:
pipeline = mlp_pipeline
X_train = X_train_ad
live_instance = live_df
model_name = 'RFC'

interpretations = interpret_lime_results_v5(pipeline, X_train, live_instance, model_name)
interpretations

Intercept 0.6237402976621272
Prediction_local [0.74565439]
Right: 0.9089398017750197


"<ul><li>The feature 'chg_amt_ratio' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'bal_chg' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'orig_zero' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'amt_bal_ratio' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'oldbalanceOrg' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'amount' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'step' is considered in the model's classification as 'Fraudulent'.</li><li>The feature 'type' is not considered a significant factor in the model's classification as 'Fraudulent'.</li><li>The model considers multiple features to assess each transaction, focusing on patterns that indicate potentially fraudulent or legitimate activity.</li></ul>"

In [42]:
pipeline = gbc_pipeline
X_train = X_train_ad
y_train = y_train_ad
live_instance = live_df
model_name = 'RFC'


cf_as_dict, original_instance =  cf_explanations(pipeline, X_train, y_train, live_instance, model_name, total_CFs=1)
narrative = explain_counterfactual_percentage(original_instance, cf_as_dict)
#cfe = cfe_live_v4(pipeline, X_train, y_train, live_instance, model_name, total_CFs=1)
#print(cfe)
print(narrative)


100%|██████████| 1/1 [00:01<00:00,  1.52s/it]

<p>To change the model's prediction, consider the following adjustments:</p><ul><li>Change <strong>chg_amt_ratio</strong> by -200.00% (from 1.0 to -1.0).</li></ul>


In [20]:
from typing import List

def cfe_live_v4(pipeline, X_train, y_train, live_instance, model_name, total_CFs=2):
    """
    Generate counterfactual explanations for a given live instance using DiCE.

    Parameters:
    - pipeline: Trained machine learning model pipeline.
    - X_train: Training data features.
    - y_train: Training data labels.
    - live_instance: The live instance for which counterfactual explanations are generated.
    - model_name: Name or identifier for the machine learning model.
    - total_CFs: Total number of counterfactuals to generate.

    Returns:
    List[dict]: List of dictionaries representing the counterfactual explanations.
    """

    # Prepare outcome data with labels
    outcome_X_train = X_train.copy()
    outcome_X_train['isFraud'] = y_train

    # Define features for DiCE
    features = ['step', 'type', 'amount', 'oldbalanceOrg', 'bal_chg', 'orig_zero',
                'amt_bal_ratio', 'chg_amt_ratio']

    # Create DiCE data object
    d = dice_ml.Data(dataframe=outcome_X_train, continuous_features=features,
                     outcome_name='isFraud')

    # Create DiCE model object
    backend = 'sklearn'
    m = dice_ml.Model(model=pipeline, backend=backend)

    # Convert live_instance to a DataFrame if it's not already
    if not isinstance(live_instance, pd.DataFrame):
        live_instance = pd.DataFrame([live_instance], columns=X_train.columns)

    print(f"Counterfactual Explanation for {model_name} using Random Method")

    exp_random = dice_ml.Dice(d, m, method='random')
    dice_exp_random = exp_random.generate_counterfactuals(live_instance, total_CFs=total_CFs, desired_class="opposite", verbose=False)

    # Extract feature names from the DiCE data object
    feature_names = d.feature_names

    # Return the counterfactuals as a list of dictionaries with feature names
    counterfactuals_with_names = []
    for cf in dice_exp_random.cf_examples_list:
        counterfactual_dict = {'feature_names': feature_names, 'feature_values': cf.final_cfs_df.iloc[0].tolist()}
        counterfactuals_with_names.append(counterfactual_dict)

    return counterfactuals_with_names


In [66]:
pipeline = rfc_pipeline
X_train = X_train_ad
y_train = y_train_ad
live_instance = live_df
model_name = 'RFC'

cf_as_dict, original_instance =  cf_explanations(pipeline, X_train, y_train, live_instance, model_name, total_CFs=1)
main_1 = main_instance(cf_as_dict, prediction, model_name)
cfe_1 = cfe_instance(original_instance, prediction)
print(main_1)
print(cfe_1)

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


AttributeError: 'list' object has no attribute 'get'

In [44]:
def cf_explanations_v1(pipeline, X_train, y_train, live_instance, model_name, total_CFs=1):
    outcome_X_train = X_train.copy()
    outcome_X_train['isFraud'] = y_train

    continuous_features = ['step', 'amount', 'oldbalanceOrg', 'bal_chg', 'amt_bal_ratio', 'chg_amt_ratio']
    d = dice_ml.Data(dataframe=outcome_X_train, continuous_features=continuous_features, outcome_name='isFraud')
    m = dice_ml.Model(model=pipeline, backend='sklearn')

    if not isinstance(live_instance, pd.DataFrame):
        live_instance = pd.DataFrame([live_instance], columns=X_train.columns)

    exp = dice_ml.Dice(d, m, method='random')
    cf = exp.generate_counterfactuals(live_instance, total_CFs=total_CFs, desired_class="opposite")

    cf_as_dict = cf.cf_examples_list[0].final_cfs_df.drop(columns=['isFraud']).iloc[0].to_dict()
    original_instance = live_instance.iloc[0].to_dict()

    return cf_as_dict, original_instance

In [67]:
cf_as_dict, original_instance = cf_explanations_v1(pipeline, X_train, y_train, live_instance, model_name, total_CFs)


100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


In [15]:
from typing import List

def cfe_live_v2(pipeline, X_train, y_train, live_instance, model_name, total_CFs=1):
    """
    Generate counterfactual explanations for a given live instance using DiCE.

    Parameters:
    - pipeline: Trained machine learning model pipeline.
    - X_train: Training data features.
    - y_train: Training data labels.
    - live_instance: The live instance for which counterfactual explanations are generated.
    - model_name: Name or identifier for the machine learning model.
    - total_CFs: Total number of counterfactuals to generate.

    Returns:
    List[dict]: List of dictionaries representing the counterfactual explanations.
    """

    # Prepare outcome data with labels
    outcome_X_train = X_train.copy()
    outcome_X_train['isFraud'] = y_train

    # Define features for DiCE
    features = ['step', 'type', 'amount', 'oldbalanceOrg', 'bal_chg', 'orig_zero',
                'amt_bal_ratio', 'chg_amt_ratio']

    # Create DiCE data object
    d = dice_ml.Data(dataframe=outcome_X_train, continuous_features=features,
                     outcome_name='isFraud')

    # Create DiCE model object
    backend = 'sklearn'
    m = dice_ml.Model(model=pipeline, backend=backend)

    # Convert live_instance to a DataFrame if it's not already
    if not isinstance(live_instance, pd.DataFrame):
        live_instance = pd.DataFrame([live_instance], columns=X_train.columns)

    print(f"Counterfactual Explanation for {model_name} using Random Method")

    exp_random = dice_ml.Dice(d, m, method='random')
    dice_exp_random = exp_random.generate_counterfactuals(live_instance, total_CFs=total_CFs, desired_class="opposite", verbose=False)
    
    # Return the counterfactuals as a list of dictionaries
    return dice_exp_random.visualize_as_list()


In [12]:
from typing import List

def cfe_live_v3(pipeline, X_train, y_train, live_instance, model_name, total_CFs=2):
    """
    Generate counterfactual explanations for a given live instance using DiCE.

    Parameters:
    - pipeline: Trained machine learning model pipeline.
    - X_train: Training data features.
    - y_train: Training data labels.
    - live_instance: The live instance for which counterfactual explanations are generated.
    - model_name: Name or identifier for the machine learning model.
    - total_CFs: Total number of counterfactuals to generate.

    Returns:
    List[dict]: List of dictionaries representing the counterfactual explanations.
    """

    # Prepare outcome data with labels
    outcome_X_train = X_train.copy()
    outcome_X_train['isFraud'] = y_train

    # Define features for DiCE
    features = ['step', 'type', 'amount', 'oldbalanceOrg', 'bal_chg', 'orig_zero',
                'amt_bal_ratio', 'chg_amt_ratio']

    # Create DiCE data object
    d = dice_ml.Data(dataframe=outcome_X_train, continuous_features=features,
                     outcome_name='isFraud')

    # Create DiCE model object
    backend = 'sklearn'
    m = dice_ml.Model(model=pipeline, backend=backend)

    # Convert live_instance to a DataFrame if it's not already
    if not isinstance(live_instance, pd.DataFrame):
        live_instance = pd.DataFrame([live_instance], columns=X_train.columns)

    print(f"Counterfactual Explanation for {model_name} using Random Method")

    exp_random = dice_ml.Dice(d, m, method='random')
    dice_exp_random = exp_random.generate_counterfactuals(live_instance, total_CFs=total_CFs, desired_class="opposite", verbose=False)
    
    # Extract feature names from the DiCE data object
    feature_names = d.feature_names

    # Return the counterfactuals as a list of dictionaries with feature names
    counterfactuals_with_names = dice_exp_random.visualize_as_list(include_feature_names=True, feature_names=feature_names)
    return counterfactuals_with_names


In [47]:
cf_as_dict, original_instance = cf_explanations(pipeline, X_train, y_train, live_df, model_name, total_CFs=1)
lime_explanation_html = interpret_lime_results_v5(pipeline, X_train, live_df, model_name)
radial_plot_html = visualize_counterfactuals_radar_plotly(cf_as_dict, original_instance)
bar_chart_html = visualize_counterfactuals_plotly(original_instance, cf_as_dict)
narrative_html = explain_counterfactual_percentage(original_instance, cf_as_dict)

print(lime_explanation_html)
print(radial_plot_html)
print(bar_chart_html)
print(narrative_html)

100%|██████████| 1/1 [00:01<00:00,  1.74s/it]


Intercept 0.11195040142422825
Prediction_local [0.12164046]
Right: 5.560618618708582e-05
<ul><li>The feature 'amt_bal_ratio' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'bal_chg' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'chg_amt_ratio' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'oldbalanceOrg' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'step' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'amount' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'orig_zero' is considered in the model's classification as 'Legitimate'.</li><li>The feature 'type' is not considered a significant factor in the model's classification as 'Legitimate'.</li><li>The model considers multiple features to assess each transaction, focusing on patterns that indicate potentially fraudulent or legitimate